This script takes the output tiles from gjengroingMetaModelDownloadImageTiles.Rmd and passes them to the meta model for canopy height inference. 

In [ ]:
import argparse
import os
import torch
from pathlib import Path
import torch.nn as nn
import rasterio
import torch.nn.functional as F
from models.backbone import SSLVisionTransformer
from models.dpt_head import DPTHead
import pytorch_lightning as pl
from models.regressor import RNet
import torchvision.transforms.functional as TF
import numpy as np